In [78]:
# Import Dependencies
! pip install splinter

import pandas as pd

from splinter import Browser
from bs4 import BeautifulSoup
import requests

import pprint

In [79]:
## Step 1A:  Scrape latest news title and paragraph text from NASA Mars News Site (Beautiful Soup)

# Set URL and retrieve page
url = "http://mars.nasa.gov/news/"
response = requests.get(url)

# Create Soup object and examine results
soup = BeautifulSoup(response.text, "html.parser")
# print(soup.prettify())

#Scrape content
news_title = soup.body.find("div", class_="content_title").a.text.strip()
news_para = soup.body.find("div", class_="rollover_description_inner").text.strip()

print(news_title)
print(news_para)

NASA Readies Perseverance Mars Rover's Earthly Twin
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


In [80]:
## Step 1B:  Find featured image (Splinter)

# set path for chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Set URL and retrieve page
url1 = "https://www.jpl.nasa.gov/spaceimages/"
browser.visit(url1)

# Navigate to large image
browser.links.find_by_partial_text("FULL IMAGE").click()
browser.links.find_by_partial_text("more info").click()
browser.find_by_tag("figure.lede").first.click()

image_url = browser.url
print(image_url)



https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA00029_hires.jpg


In [81]:
## Step 1C:  Scrape facts about Mars (Pandas)

# Set URL and retrieve page
url2 = "http://space-facts.com/mars/"
# response = requests.get(url)

mars_tables = pd.read_html(url2)[0]
mars_tables

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [83]:
##Step 1C (cont): Separate tables and clean -- mars_facts
mars_facts = mars_tables
mars_facts.columns = ["Planet Profile", "Planet Data"]
mars_facts.style.hide_index()

Planet Profile,Planet Data
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [85]:
# Render table as HTML table

html = mars_facts.to_html()
print(html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Planet Profile</th>
      <th>Planet Data</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8<

In [60]:
##Step 1C (cont): Separate tables and clean -- mars_earth
#mars_earth = mars_tables[1]
#mars_earth.columns = ["Mars-Earth Comparison", "Mars", "Earth"]
#mars_earth.style.hide_index()

In [65]:
# Create list of dictionaries to store mars_facts

keys = mars_tables[0][0]
values = mars_tables[0][1]

facts = dict(zip(keys, values))
facts.items()


# NOTE:  This list to be a list of dictionaries for Mongo Mars["facts"] collection; do I want this to be a 
# a single dictionary?  See hemisphere_image_url

dict_items([('Equatorial Diameter:', '6,792 km'), ('Polar Diameter:', '6,752 km'), ('Mass:', '6.39 × 10^23 kg (0.11 Earths)'), ('Moons:', '2 (Phobos & Deimos)'), ('Orbit Distance:', '227,943,824 km (1.38 AU)'), ('Orbit Period:', '687 days (1.9 years)'), ('Surface Temperature:', '-87 to -5 °C'), ('First Record:', '2nd millennium BC'), ('Recorded By:', 'Egyptian astronomers')])

In [7]:
## Step 1C (cont): Convert dat to HTML strings
facts_html = mars_facts.to_html()
compare_html = mars_earth.to_html()

In [62]:
## Step 1D:  Obtain high resolution images for each of Mars' hemispheres

# set path for chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Set URL, visit page, inspect HTML code
url3 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
response = requests.get(url3)
browser.visit(url3)

# Create Soup object, grab each hemisphere division  
soup = BeautifulSoup(response.text, "html.parser")
images = soup.find_all("div", class_="item")

# Print and review image 
print(len(images))
print(images[0])


4
<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/><div class="description"><h3>Cerberus Hemisphere Enhanced</h3></div></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>


In [63]:
# Loop through each soup object ("images"); use splinter to navigate to full res image URL; store URL and name 
# NOTE:  because HREF stored a partial href, it will be appended the base USGS url

hemi_title =[]
hemi_url =[]
hemisphere_image_url=[]

for image in images:
    
    # Extract title and append to hemi_title
    title = image.h3.text
    hemi_title.append(title)
    
    #Extract href from image and append to base URL to get to image file
    base_url = "https://astrogeology.usgs.gov"
    full_url = base_url+image.a["href"]

    # Create soup object and extract full image URL; append to hemi_url
    response1 = requests.get(full_url)
    soup = BeautifulSoup(response1.text, "html.parser")
    image_url = soup.find("div", class_="downloads").a["href"]
    hemi_url.append(image_url)
    
    # Create Hemisphere Image list
    hemisphere_image_url.append({"title": title, "image_url": image_url})
      
    print(title)
    print(image_url)
    print("-------------------------------------------------------------------------------------------------")
    
pprint.pprint(hemisphere_image_url)

# NOTE:  hemisphere_image_url to be list of dictionaries for Mongo Mars["images"] collection

Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
-------------------------------------------------------------------------------------------------
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
-------------------------------------------------------------------------------------------------
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
-------------------------------------------------------------------------------------------------
Valles Marineris Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
-------------------------------------------------------------------------------------------------
[{'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_e

In [67]:
hemisphere_image_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [32]:
# Alternative search for this info

# Create dictionary
hemisphere_image_url={}
titles =[]

# set path for chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Set URL, visit page, inspect HTML code
url3 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
response = requests.get(url3)
browser.visit(url3)

# Create Soup object, grab each hemisphere division  
soup = BeautifulSoup(response.text, "html.parser")
#images = soup.find_all("div", class_="item")

titles = soup.find_all("div", class_="item").h3.text

AttributeError: ResultSet object has no attribute 'h3'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?